# Benchmark Models

This notebook implements simple benchmark models to compare against LSTM performance:

In [ ]:
#| default_exp benchmark_model

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.basics import patch
from pathlib import Path
import hvplot.pandas
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
#| export
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
#| hide
#| eval: false
DATA = Path("../../data")

In [ ]:
#| hide
#| eval: false
df = pd.read_csv(
    DATA/'data_cumul.csv', 
    sep=';', 
    usecols=['time', 'P_mean', 'P_cumul_7j', 'débit_insitu', 'débit_mgb'], 
    index_col='time',
    converters={"time": pd.to_datetime}
    )

In [ ]:
#| export
def normalize(df):
    return (df - df.min()) / (df.max() - df.min())

In [ ]:
#| hide
#| eval: false
normalize(df).plot()#.hvplot.line()

#### Select feature and target columns

In [ ]:
#| hide
#| eval: false
x_col, y_col = ['P_cumul_7j','débit_mgb'], ['débit_insitu']

#### Scale data

In [ ]:
#| hide
#| eval: false
from sklearn.preprocessing import RobustScaler

features_scaler = RobustScaler()
features = df[x_col]
df[x_col] = features_scaler.fit_transform(features)

### Split data

In [ ]:
#| hide
#| eval: false
train_mask = df.index < '2019-01-01'
train = df[train_mask]
valid = df[~train_mask]

### Feature generator
> Feature generator that uses a sliding window of n previous timesteps and polynomial features to predict the next value, enabling learning of temporal patterns and non-linear relationships

In [ ]:
#| export
class FeatureGenerator:
    """
    Transforms time series data into feature matrices suitable for machine learning models.
    Creates lagged features using a sliding window and optionally generates polynomial features
    to capture non-linear relationships between variables.
    """
    def __init__(self, context_window: int = 10, target_window: int = 10, degree: int = 1):
        self.context_window = context_window
        self.target_window = target_window
        self.poly_features = PolynomialFeatures(degree=degree)
        
    def generate(self, df: pd.DataFrame, x_col: list[str], y_col: list[str]) -> tuple[pd.DataFrame, pd.DataFrame]:
        X, y = df[x_col], df[y_col]
        if 1 < self.poly_features.degree:
            X = pd.DataFrame(self.poly_features.fit_transform(X), index=X.index)
        X, y = self.generate_sliding_window_features(X, y)
        return X, y


    def generate_sliding_window_features(self, X: pd.DataFrame, y: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
        """
        Creates a feature matrix by combining multiple input variables and their lagged values.
        For each time step t, takes values from t-window to t for each input variable
        and combines them into a single feature vector. The target value is taken at time t.
        This allows the model to learn patterns across multiple timesteps.
        """
        features = []
        targets = []
        
        for i in range(len(X) - self.context_window - self.target_window):
            row_features = X.iloc[i:i + self.context_window]
            features.append(row_features.values.reshape(-1))
            row_targets = y.iloc[i + self.context_window: i + self.context_window + (self.target_window + 1)]
            targets.append(row_targets.values.reshape(-1))

        features = pd.DataFrame(index=X.index[self.context_window:len(X) - self.target_window], data=features)
        targets = pd.DataFrame(
            index=y.index[self.context_window:len(X) - self.target_window],
            data=targets,
            columns=[f"t+{i}" for i in range(0, self.target_window+1)])
        return features, targets
    


In [ ]:
#| hide
#| eval: false
feature_generator = FeatureGenerator(1, 0)
train_x, train_y = feature_generator.generate(train, x_col, y_col)


### Benchmark model
We chose a linear regression model as our benchmark because:
1. When combined with our feature generator, it provides a simple yet effective baseline
2. The polynomial features allow it to capture non-linear relationships in the data
3. The sliding window enables learning of temporal patterns across multiple timesteps
4. Linear regression models are highly interpretable and computationally efficient

In [ ]:
#| export
class SimpleRegressionModel:
    def __init__(self):
        self.model = LinearRegression()
    
    def fit(self, X, y=None):
        self.model.fit(X, y)
        return self
    
    def predict(self, X):
        pred = self.model.predict(X)
        return pd.DataFrame(pred, index=X.index, columns=[f"t+{i}" for i in range(0, pred.shape[1])])

### Model training and comparison

In [ ]:
#| hide
#| eval: false
predictions = []
scores = []
for degree in range(1, 4):
    for window in range(10, 51, 10):
        feature_generator = FeatureGenerator(context_window=window, target_window=10, degree=degree)        
        train_x, train_y = feature_generator.generate(train, x_col, y_col)
        valid_x, valid_y = feature_generator.generate(valid, x_col, y_col)

        model = SimpleRegressionModel()
        model.fit(train_x, train_y)
        
        pred = model.predict(valid_x)
        pred_df = pred.copy()
        pred_df['degree'] = degree
        pred_df['window'] = window
        pred_df = pred_df.set_index(['degree', 'window'], append=True)
        predictions.append(pred_df)

predictions_ds = pd.concat(predictions).reorder_levels(['degree', 'window', 'time']).to_xarray()
observations = valid[y_col].to_xarray().sel(time=slice(predictions_ds.time.min(), predictions_ds.time.max()))
results_ds = predictions_ds.merge(observations)

## Scoring

Since we have generated multiple predictions with different parameters, we will select only the best performing models according to each metric

In [ ]:
#| export
class BenchmarkScores:
    """
    A class to compute and compare different error metrics between predictions and observations.
    
    This class provides a flexible way to compute multiple error metrics between predicted and observed values.
    New metrics can be easily added by implementing them as methods.
    The new metrics can then be used by passing their method names as strings to compute_scores().
    
    Example:

    ```python
    class CustomBenchmarkScores(BenchmarkScores):
        def mse(self, pred, obs):
            error = ((pred - obs)**2).mean(dim="time") 
            error.name = "mse"
            return error
        
    # Can then be used as:
    scores = compute_scores(pred, obs, metrics=["mae", "rmse", "mse"])
    ```
    """
    def __init__(self):
        pass

    def compute_scores(self, predictions: xr.DataArray, observations: xr.DataArray, metrics: list[str]):
        """Compute the scores for the predictions and observations."""
        scores = []
        for metric in metrics:
            scores.append(getattr(self, metric)(predictions, observations).to_dataset(name=metric))
        return xr.merge(scores)
    
    def mae(self, pred, obs):
        error = abs(pred - obs).mean(dim="time")
        error.name = "mae"
        return error

    def rmse(self, pred, obs):
        error = np.sqrt(((pred - obs)**2).mean(dim="time"))
        error.name = "rmse"
        return error
    
    def nse(self, pred, obs):
        """
        Calculate Nash-Sutcliffe Efficiency score.
        
        $NSE = 1 - \\frac{\\sum(pred - obs)^2}{\\sum(obs - \\overline{obs})^2}$
        
        NSE ranges from -inf to 1, with 1 being a perfect match
        """
        numerator = ((pred - obs)**2).sum(dim="time")
        denominator = ((obs - obs.mean(dim="time"))**2).sum(dim="time")
        error = 1 - (numerator / denominator)
        error.name = "nse"
        return error
    
    def kge(self, pred, obs):
        """Calculate Kling-Gupta Efficiency score.
        
        $KGE = 1 - sqrt((r-1)^2 + (\alpha-1)^2 + (\beta-1)^2)$
        
        where:
        - r = Pearson correlation coefficient
        - $\\alpha = \\frac{std(pred)}{std(obs)}$ 
        - $\\beta = \\frac{mean(pred)}{mean(obs)}$
        
        KGE ranges from -inf to 1, with 1 being a perfect match
        """
        # Calculate components
        r = xr.corr(pred, obs, dim="time")
        alpha = pred.std(dim="time") / obs.std(dim="time")
        beta = pred.mean(dim="time") / obs.mean(dim="time")
        
        # Calculate KGE
        error = 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
        error.name = "kge"
        return error

    def find_nbest_scores(
            self,
            ds: xr.Dataset, # Dataset containing scores as variables
            how: dict[str, str], # Dictionary of how to find the best scores for each variable
            n: int=2 # Number of best scores to find
        ):
        """Given a dataset containing scores as variables,find the n best scores for each score."""
        assert all(how in ["min", "max"] for how in how.values()), "how must be either 'min' or 'max'"
        df = ds.to_dataframe()
        df = df.reorder_levels(["variable"] + [level for level in df.index.names if level != "variable"])
        best_scores_coords = []
        for score, how in how.items():
            ascending = how == "min"
            best_variable_scores = self._find_best_scores_by_variable(df[score], n, ascending)
            best_scores_coords.extend(best_variable_scores.values.tolist())
        best_scores_coords = pd.MultiIndex.from_tuples(best_scores_coords, names=df.index.names)
        n_best_scores = df.loc[best_scores_coords]
        n_best_scores = n_best_scores.sort_index().drop_duplicates()
        return n_best_scores

    def _find_best_scores_by_variable(self, serie: pd.Series, n, ascending: bool):
        """For each unique variable in a multi-indexed Series, find indices of the n best values based on ascending/descending sort."""
        sorted = serie.sort_values(ascending=ascending)
        nbest_values = []
        for v in sorted.index.get_level_values("variable").unique():
            nbest_values.append(sorted.loc[[v]].head(n))
        nbest_values = pd.concat(nbest_values)
        return nbest_values.index

In [ ]:
#| hide
#| eval: false
benchmark_scores = BenchmarkScores()
scores_ds = benchmark_scores.compute_scores(
    predictions_ds.to_array(),
    observations["débit_insitu"],
    ["mae", "rmse", "nse", "kge"])
best_scores = benchmark_scores.find_nbest_scores(
    scores_ds,
    how={"mae": "min", "rmse": "min", "nse": "max", "kge": "max"},
    n=1)

#### Static plotting

Finally we will plot the error so we can visualize the best results

In [ ]:
#| export
def plot_static_benchmark_scores(
        df: pd.DataFrame, # Dataframe with polynomial degree and window as index and mae and mse as columns
        figsize: tuple=(8, 7), # Figure size in inches (width, height)
        fontsize: int=7, # Font size for annotations
        xlim: tuple=(None, None), # Tuple of (min, max) values for x-axis limits
        ylim: tuple=(None, None), # Tuple of (min, max) values for y-axis limits
        ):
    """Plot MAE vs MSE scores with degree and window annotations for model comparison"""
    fig, ax = plt.subplots(figsize=figsize)
    
    # Get unique time steps - expected format 't+N' where N is 0-10
    time_steps = []
    for col in df.index.get_level_values(0).unique():
        if isinstance(col, str) and col.startswith('t+'):
            time_steps.append(col)
    time_steps = sorted(time_steps, key=lambda x: int(x.split('+')[1]))
    
    # Create a colormap
    colors = plt.cm.viridis(np.linspace(0, 1, len(time_steps)))
    
    # Plot each time step with different color
    for t, color in zip(time_steps, colors):
        mask = df.index.get_level_values(0) == t
        df_t = df[mask]
        scatter = ax.scatter(df_t['mae'], df_t['rmse'], label=t, color=color)
        
        # Add annotations for each point
        for (_, deg, win), (mae, rmse) in zip(df_t.index, df_t[['mae', 'rmse']].values):
            ax.annotate(f'd={deg},w={win}', (mae, rmse), 
                       xytext=(5, 5), textcoords='offset points', 
                       fontsize=fontsize, color=color)

    ax.set(xlabel='MAE', ylabel='RMSE', 
           title='MAE vs RMSE for different degrees, windows and prediction horizons')
    ax.legend(title='Prediction horizon', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xlim(*xlim)
    plt.ylim(*ylim)
    plt.tight_layout()
    plt.show()

In [ ]:
#| hide
#| eval: false
plot_static_benchmark_scores(best_scores)

#### Interactive plotting

In [ ]:
#| export
def plot_interactive_benchmark_scores(
        df: pd.DataFrame, # Dataframe with polynomial degree and window as index and mae and mse as columns
        figsize: tuple=(800, 600), # Figure size in pixels (width, height)
        fontsize: int=12, # Font size for annotations
        xlim: tuple=(None, None), # Tuple of (min, max) values for x-axis limits
        ylim: tuple=(None, None), # Tuple of (min, max) values for y-axis limits
        ):
    """Plot MAE vs MSE scores with degree and window annotations for model comparison using Plotly"""
    import plotly.graph_objects as go
    import plotly.express as px
    
    # Get unique time steps - expected format 't+N' where N is 0-10
    time_steps = []
    for col in df.index.get_level_values(0).unique():
        if isinstance(col, str) and col.startswith('t+'):
            time_steps.append(col)
    time_steps = sorted(time_steps, key=lambda x: int(x.split('+')[1]))
    # Create a colormap
    # Generate more colors by interpolating the Viridis colormap
    n_colors = len(time_steps)
    colors = px.colors.sample_colorscale('Viridis', n_colors)
    # Create figure
    fig = go.Figure()
    
    # Plot each time step with different color
    for t, color in zip(time_steps, colors):
        mask = df.index.get_level_values(0) == t
        df_t = df[mask]
        
        # Add scatter plot
        fig.add_trace(go.Scatter(
            x=df_t['mae'],
            y=df_t['rmse'],
            mode='markers+text',
            name=t,
            marker=dict(color=color, size=10),
            text=[f'd={deg},w={win}' for (_, deg, win) in df_t.index],
            textposition="top center",
            textfont=dict(size=fontsize),
            hovertemplate="MAE: %{x:.3f}<br>RMSE: %{y:.3f}<br>%{text}<extra></extra>"
        ))
    
    # Update layout
    fig.update_layout(
        width=figsize[0],
        height=figsize[1],
        title='MAE vs RMSE for different degrees, windows and prediction horizons',
        xaxis_title='MAE',
        yaxis_title='RMSE',
        legend_title='Prediction horizon',
        hovermode='closest',
        showlegend=True
    )
    
    # Set axis limits if provided
    if xlim[0] is not None or xlim[1] is not None:
        fig.update_xaxes(range=xlim)
    if ylim[0] is not None or ylim[1] is not None:
        fig.update_yaxes(range=ylim)
    
    return fig


In [ ]:
#| hide
#| eval: false
plot_interactive_benchmark_scores(best_scores,)


Based on the scatter plot comparing MAE vs MSE metrics, we can conclude that polynomial regression with degree 2 and window sizes between 30-50 days provides the optimal predictions. This is evident from the cluster of points in the lower left corner of the plot, which indicates lower error rates for both metrics. Specifically, the combinations of degree=2 with windows around 40 days achieve the best balance between Mean Absolute Error and Mean Squared Error, suggesting these parameters offer the most accurate and stable predictions without overfitting the data.

## Save prediction data

We save the best model for later use

In [ ]:
#| hide
#| eval: false
benchmark_ds = results_ds.sel(degree=2, window=slice(30,50))
benchmark_ds.to_netcdf(DATA/'regression_benchmark.nc')

## Time series verification

Now that we have identified the optimal model parameters, let's verify its performance by comparing the predicted discharge values with both observed values and MGB model predictions. This comparison will be done across the full 10-day prediction horizon to assess how well our model maintains its predictive power over time. We'll visualize these comparisons using time series plots that show the observed discharge, our model's predictions, and the MGB model predictions side by side.


In [ ]:
#| export
def plot_prediction_comparison(
        observed: xr.DataArray, # Time series of observed water discharge values
        predicted: xr.Dataset, # Dataset containing predicted discharge values for each forecast horizon (t+1 to t+10)
        mgb: xr.DataArray # Time series of MGB model water discharge predictions
        ) -> plt.Figure:
    """Plot comparison between observed, predicted and MGB discharge values for a 10-day horizon."""
    fig, axes = plt.subplots(5, 2, figsize=(14, 10), sharex=True, sharey=True)
    axes = axes.flatten()

    for i in range(0, 10):
        y_obs = observed.to_series()
        y_pred = predicted[f"t+{i+1}"].to_series()
        y_mgb = mgb.to_series()
        
        rmse = np.sqrt(((y_obs - y_pred)**2).mean())
        correlation = np.corrcoef(y_obs, y_pred)[0, 1]

        axes[i].plot(y_obs, label="Q_obs", color='blue')
        axes[i].plot(y_pred, label="Q_pred", color='red', linestyle='solid')
        axes[i].plot(y_mgb, label="Q_mgb", color='black')

        axes[i].set_title(f"Jour {i+1} RMSE={rmse:.2f}, Corr={correlation:.2f}")
        
        axes[i].grid()
        axes[i].legend()

    plt.suptitle("Comparaison des valeurs réelles et prédites pour chaque jour de l'horizon de 10 jours", fontsize=14)
    plt.tight_layout()
    return fig

We first need to reescale the MGB data

In [ ]:
#| hide
#| eval: false
rescaled_features = pd.DataFrame(features_scaler.inverse_transform(valid[x_col]), index=valid.index, columns=x_col)
debit_mgb = rescaled_features["débit_mgb"].to_xarray()

In [ ]:
#| hide
#| eval: false
fig = plot_prediction_comparison(
    observed=results_ds["débit_insitu"],
    predicted=results_ds.sel(degree=1, window=50), 
    mgb=debit_mgb)
plt.show()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()